In [1]:
!pwd

/home/mchistya/code/eangelid/trampact/notebooks


In [2]:
!pip install lambert

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from lambert import Lambert93, convertToWGS84Deg

# from iris import get_iris_type
# add iris type to data on T1
# add t2 on data 2008 ?

In [4]:
import sys
for iris in sys.path:
    print(iris)

/home/mchistya/code/eangelid/trampact/notebooks
/home/mchistya/code/mchistya/data-challenges/04-Decision-Science
/home/mchistya/code/eangelid/trampact/notebooks
/home/mchistya/.pyenv/versions/3.8.6/lib/python38.zip
/home/mchistya/.pyenv/versions/3.8.6/lib/python3.8
/home/mchistya/.pyenv/versions/3.8.6/lib/python3.8/lib-dynload

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages
/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/IPython/extensions
/home/mchistya/.ipython


# first dataset T1

In [5]:
data = pd.read_csv('../raw_data/BPE_T1.csv')


## Dataset properties

Note sur Lambert
L'expression des coordonnées dans ce système est tridimensionnelle sous forme de longitudes, latitudes et hauteurs ellipsoïdales. 

In [6]:
data.head(5)

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE
0,93,6,6088,06088_0801,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport
1,93,6,6088,06088_1005,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé
2,93,6,6088,06088_1005,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme
3,93,6,6088,06088_3801,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a
4,93,6,6088,06088_3801,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a


In [7]:
pd.unique(data['DEPCOM'])

array([6088, 6159])

In [8]:
data.shape

(11857, 11)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   REG         11857 non-null  int64  
 1   DEP         11857 non-null  int64  
 2   DEPCOM      11857 non-null  int64  
 3   DCIRIS      11857 non-null  object 
 4   AN          11857 non-null  int64  
 5   TYPEQU      11857 non-null  object 
 6   LAMBERT_X   11857 non-null  float64
 7   LAMBERT_Y   11857 non-null  float64
 8   QUALITE_XY  11857 non-null  object 
 9   LAB         11857 non-null  object 
 10  CLASSE      11857 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1019.1+ KB


In [10]:
features = list(data.columns)
features;

## Transform iris code to integer value

In [11]:
data.head(1)

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE
0,93,6,6088,06088_0801,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport


In [12]:
data['iris_id']=data['DCIRIS'].map(lambda x: x[0:5]+x[6:])

In [13]:
data['iris_id'];

In [14]:
# drop  inconveniant iris code

data.drop(columns=['DCIRIS','REG','DEP','DEPCOM'],inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AN          11857 non-null  int64  
 1   TYPEQU      11857 non-null  object 
 2   LAMBERT_X   11857 non-null  float64
 3   LAMBERT_Y   11857 non-null  float64
 4   QUALITE_XY  11857 non-null  object 
 5   LAB         11857 non-null  object 
 6   CLASSE      11857 non-null  object 
 7   iris_id     11857 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 741.2+ KB


In [15]:
pd.unique(data['AN']) # observations only for 2008

array([2018])

In [16]:
pd.unique(data['QUALITE_XY']) # coordinates quality of X and Y

array(['Bonne', 'Mauvaise', 'Acceptable'], dtype=object)

In [17]:
data['QUALITE_XY'].value_counts()

Bonne         11598
Mauvaise        157
Acceptable      102
Name: QUALITE_XY, dtype: int64

## Geographic to cartesian transformation

The goal is to convert RGF93 Lambert 93 into WGS84 (world geographic system 1984), - to get the longitude and the latitude

Documentation sur le package

https://pypi.org/project/lambert/

In [18]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801


In [19]:
# create coordinates input list

lambert_x = list(data['LAMBERT_X'])
lamber_y = list(data['LAMBERT_Y'])
coord = zip(lambert_x,lamber_y)

In [20]:
longitude = []
latitude = []
for x, y in coord:
    pt = convertToWGS84Deg(x, y, Lambert93)
    longitude.append(pt.getX())
    latitude.append(pt.getY())

In [21]:
data['longitude']=longitude

In [22]:
data['latitude']=latitude

In [23]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id,longitude,latitude
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801,7.321389,43.723333
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005,7.278395,43.739469
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005,7.278369,43.739746
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801,7.269413,43.741919
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801,7.258674,43.742321


In [24]:
data.to_csv('../raw_data/data_provis.csv')

### Encode Quality_XY

In [25]:
enc = OneHotEncoder(sparse=False)

In [26]:
enc.fit(data[['QUALITE_XY']])

OneHotEncoder(sparse=False)

In [27]:
pd.unique(data['QUALITE_XY'])

array(['Bonne', 'Mauvaise', 'Acceptable'], dtype=object)

In [28]:
quality_xy_encoded = enc.transform(data[['QUALITE_XY']])
quality_xy_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [29]:
data["Q_XY_acceptable"],data["Q_XY_bonne"],data['Q_XY_mauvaise']=quality_xy_encoded.T

In [30]:
data['Q_XY_bonne'].value_counts()

1.0    11598
0.0      259
Name: Q_XY_bonne, dtype: int64

In [31]:
data['Q_XY_mauvaise'].value_counts()

0.0    11700
1.0      157
Name: Q_XY_mauvaise, dtype: int64

In [32]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id,longitude,latitude,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801,7.321389,43.723333,0.0,1.0,0.0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005,7.278395,43.739469,0.0,1.0,0.0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005,7.278369,43.739746,0.0,1.0,0.0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801,7.269413,43.741919,0.0,1.0,0.0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801,7.258674,43.742321,0.0,0.0,1.0


### Numerical features

In [33]:
features_num = ['REG',
 'DEP',
 'DEPCOM',
 'iris',
 'AN','LAMBERT_X',
 'LAMBERT_Y']

In [34]:
data['TYPEQU']

0        F106
1        D233
2        G104
3        A505
4        A505
         ... 
11852    D237
11853    D242
11854    D301
11855    E101
11856    E101
Name: TYPEQU, Length: 11857, dtype: object

In [35]:
# number of different iris => not only irises with the tramway line
len(pd.unique(data['iris_id']))

110

### Categorical features

In [36]:
features_cat = ['LAB','CLASSE','QUALITE_XY','TYPEQU']
features_cat

['LAB', 'CLASSE', 'QUALITE_XY', 'TYPEQU']

In [37]:
data['iris_id'].value_counts()

060882203    873
060882201    725
060882202    435
060882401    414
060882402    336
            ... 
060882805      1
060882703      1
060880903      1
060882704      1
060882902      1
Name: iris_id, Length: 110, dtype: int64

In [38]:
len(pd.unique(data['LAMBERT_X']))

4915

In [39]:
len(pd.unique(data['LAMBERT_Y']))

4866

In [40]:
data[features_cat]

,LAB,CLASSE,QUALITE_XY,TYPEQU
0,Centre équestre,Sport,Bonne,F106
1,Masseur kinésithérapeute D235,Santé,Bonne,D233
2,Information touristique,Tourisme,Bonne,G104
3,Agence immobilière,Services a,Bonne,A505
4,Agence immobilière,Services a,Mauvaise,A505
...,...,...,...,...
11852,Pédicure,Santé,Bonne,D237
11853,Diététicien,Santé,Bonne,D242
11854,Pharmacie,Santé,Bonne,D301
11855,Taxi,Transport,Bonne,E101


In [41]:
pd.unique(data['LAB'])

array(['Centre équestre', 'Masseur kinésithérapeute D235',
       'Information touristique', 'Agence immobilière',
       'Magasin de vêtements', 'Établissement santé court séjour',
       'Établissement santé moyen séjour',
       'Établissement santé long séjour', 'Psychologue', 'Pharmacie',
       'Hôtel', "Banque, Caisse d'épargne", 'Pompes funèbres',
       'Bureau de poste', 'Réparation auto et de matériel agicole',
       'Contrôle technique automobile', 'École de conduite',
       'Électricien', 'Entreprise générale du bâtiment', 'Coiffure',
       'Vétérinaire', 'Restaurant', 'Pressing', 'Institut de beauté',
       'Supermarché', 'Grande surface de bricolage', 'Supérette',
       'Épicerie', 'Boulangerie', 'Boucherie charcuterie',
       'Librairie papeterie journaux', "Magasin d'équipements du foyer",
       'Magasin de chaussures',
       "Magasin d'électroménager, audio vidéo B306",
       'Magasin de meubles',
       "Magasin d'art. de sports

In [42]:
data['LAB'].value_counts(ascending=True);

In [43]:
len(pd.unique(data['TYPEQU'])) # unique values of equipement type

152

In [44]:
data['TYPEQU'].value_counts(); # for meaning refer to 'LAB'

In [45]:
len(pd.unique(data['LAB'])) # Equipement label

152

In [46]:
len(pd.unique(data['CLASSE']))

7

In [47]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id,longitude,latitude,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801,7.321389,43.723333,0.0,1.0,0.0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005,7.278395,43.739469,0.0,1.0,0.0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005,7.278369,43.739746,0.0,1.0,0.0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801,7.269413,43.741919,0.0,1.0,0.0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801,7.258674,43.742321,0.0,0.0,1.0


### Devision of LAB in categories

Produce a modification of CLASSE variable to improve grouping and compare with ML clustering

In [48]:
# Size of categories

data['CLASSE'].value_counts() # pb of imbalance

Services a    5263
Santé         3125
Commerces     2403
Transport      443
Tourisme       310
Enseigneme     200
Sport          113
Name: CLASSE, dtype: int64

In [49]:
pd.unique(data['LAB'][data['CLASSE']=='Sport']);

In [50]:
cat_sport =['Athlétisme', 'Salles de remise en forme',
       'Salles multisports (gymnase)', 'Salles spécialisées',
       'Salles de combat', 'Bassin de natation',
       'Plateaux et terrains de jeux extérieurs F112',
       'Terrain de grands jeux', 'Sports nautiques', 'Tennis', 'Roller', 'Salles non spécialisées',
       'Sports de glace', 'Boulodrome']
cat_leasure = ['Centre équestre', 'Théâtre', 'Port de plaisance', 'Musée', 'Cinéma', 'Bowling']

In [51]:
pd.unique(data['LAB'][data['CLASSE']=='Santé']);

In [52]:
cat_medical = ['Masseur kinésithérapeute D235',
       'Établissement santé court séjour',
       'Établissement santé moyen séjour',
       'Établissement santé long séjour', 'Psychologue', 'Pharmacie',
       'Centre de santé', 'Dialyse', 'Médecin généraliste',
       'Spécialiste en cardiologie',
       'Spécialiste en dermatologie vénéréologie D206',
       'Spécialiste en psychiatrie', 'Spécialiste en ophtalmologie',
       'Spécialiste en oto',
       'Spéc. en radiodiagnostic et imagerie médicale D213',
       'Spécialiste en gynécologie (médicale et/ou obstétrique)',
       'Chirurgien dentiste', 'Sage', 'Infirmier', 'Orthophoniste',
       'Orthoptiste', 'Diététicien',
       'Enfants handicapés.: soins à domicile',
       'Adultes handicapés : services d’aide',
       'Spécialiste en pédiatrie', 'Audio prothésiste',
       "Laboratoire d'analyses et de biologie médicales", 'Ambulance',
       'Ergothérapeute', 
       'Établissement psychiatrique avec hébergement D105',
       'Structures psychiatriques en ambulatoire D110', 'Crèche',
       'Spécialiste en gastro-entérologie', 'Spécialiste en pneumologie',
       'Spécialiste en stomatologie',
       "Personnes âgées : services d'aide",
       'Personnes âgées : soins à domicile', 'Psychomotricien',
       'Adultes handicapés : hébergement', 'Centre médecine préventive',
       'Centre lutte cancer', 'Urgences',
       'Enfants handicapés : hébergement', 'Travail protégé',
       'Maternité',
       'Hospitalisation à domicile',
       'Adultes handicapés : services de soins à domicile']

cat_socio_med = ['Personnes âgées : hébergement',
                 'Autres établissements pour adultes et familles en difficulté',
                 "Aide sociale à l'enfance : action éduc.",
                 "CHRS Centre d'héberg. et de réadapt. sociale",
                 'Personnes âgées : foyers restaurants',
                 "Aide sociale à l'enfance : hébergement",
                  "Centre accueil demandeur d'asile"]

In [53]:
pd.unique(data['LAB'][data['CLASSE']=='Tourisme']);

In [54]:
cat_tourist = ['Information touristique', 'Hôtel', 'Agence de voyages']

In [55]:
pd.unique(data['LAB'][data['CLASSE']=='Services a']);

In [56]:
cat_general_service = ['Bureau de poste',                   
       'Réseau partenarial pôle emploi',
       'Direction Régionale des Finances Publiques (DRFIP)',
       'Direction Départementale des Finances Publiques (DDFIP)',
       'Agence de travail temporaire', 'Réseau spécialisé Pôle emploi',
       'Réseau de proximité Pôle Emploi', 'Police',
       'Tribunal de grande instance (TGI)', "Tribunal d'instance (TI)",
       "Conseil départemental d'accès au droit (CDAD)", 'Gendarmerie',
       "Conseil des prud'hommes (CPH)", 'Relais poste',
       'Tribunal de commerce (TCO)']

In [57]:
pd.unique(data['LAB'][data['CLASSE']=='Commerces']);

In [58]:
cat_commerce = ['Supermarché', 'Supérette', 'Épicerie',
       'Boulangerie', 'Boucherie charcuterie',
       'Librairie papeterie journaux', "Magasin d'équipements du foyer",
       
       "Magasin d'électroménager, audio vidéo B306",      
       "Magasin d'art. de sports et de loisirs B308",
       'Droguerie, quincaillerie - bricolage', 
       'Magasin de matériel médical et orthopédique', 'Poissonnerie',
       'Produits surgelés',
               'Pompes funèbres',
               'Électricien', 'Vétérinaire', 'Pressing']

cat_commertce_x = ['Agence immobilière', "Banque, Caisse d'épargne", 'Coiffure', 'Restaurant',
                  'Institut de beauté', 'Pédicure',
                  'Magasin de vêtements', 'Magasin de chaussures',
                  'Parfumerie', 'Horlogerie Bijouterie', 
                   'Fleuriste', 'Magasin d’optique',]

cat_grande_surface = ['Grande surface de bricolage', 'Magasin de meubles', 
                      'Station service', 'Magasin de revêtements murs et sols', 'Hypermarché',
                     'Réparation auto et de matériel agicole', 'Contrôle technique automobile',
                     'Entreprise générale du bâtiment']

In [59]:
pd.unique(data['LAB'][data['CLASSE']=='Enseigneme']);

In [60]:
cat_high_educ = [ 'Autre formation post bac non universitaire',
                 'Formation commerce',
                 'Enseignement général supérieur privé', 'STS CPGE', 'UFR',
                 'Institut universitaire',
                 'Enseignement général supérieur privé', 'UFR',
                 'Institut universitaire',
                 'Autre enseignement supérieur', 'Résidence universitaire',
                 'Restaurant universitaire']

cat_schools = ['École élémentaire', 'Collège',
       "Lycée d'enseignement professionnel",
       "Lycée d'enseignement général et/ou techno.",
       'École maternelle',
       'Formation santé', 'Formation commerce',
       'STS CPGE', 
       "Centre de formation d'apprentis (hors agriculture)",
       'SEP : Section enseignement professionnel',
              'École de conduite',]

In [61]:
pd.unique(data['LAB'][data['CLASSE']=='Transport']);

In [62]:
cat_transport = ['Taxi', 'Gare de voyageurs d’intérêt national E108',
       'Gare de voyageurs d’intérêt local',
       'Gare de voyageurs d’intérêt régional',
                'Location auto-utilitaire légers',]

### Create columns for each category

In [63]:
def is_sport(cat):
    return cat.map(lambda x: 1 if x in  cat_sport else 0)

def is_leasure(cat):
    return cat.map(lambda x: 1 if x in  cat_leasure else 0)

def is_medical(cat):
    return cat.map(lambda x: 1 if x in  cat_medical else 0)

def is_socio_med(cat):
    return cat.map(lambda x: 1 if x in  cat_socio_med else 0)

def is_tourist(cat):
    return cat.map(lambda x: 1 if x in  cat_tourist else 0)

def is_general_service(cat):
    return cat.map(lambda x: 1 if x in  cat_general_service else 0)

def is_commerce(cat):
    return cat.map(lambda x: 1 if x in  cat_commerce else 0)

def is_commerce_spec(cat):
    return cat.map(lambda x: 1 if x in  cat_commertce_x else 0)

def is_grande_surface(cat):
    return cat.map(lambda x: 1 if x in  cat_grande_surface else 0)

def is_high_educ(cat):
    return cat.map(lambda x: 1 if x in  cat_high_educ else 0)

def is_schools(cat):
    return cat.map(lambda x: 1 if x in  cat_schools else 0)

def is_transport(cat):
    return cat.map(lambda x: 1 if x in  cat_transport else 0)

In [64]:
data_cat = data.agg({
    'LAB':[is_sport, is_leasure, is_medical, is_socio_med, 
           is_tourist,is_general_service,is_commerce,is_commerce_spec,
           is_grande_surface,is_high_educ,is_schools,is_transport]
})

In [65]:
data_cat.head()

LAB                                                                   \
  is_sport is_leasure is_medical is_socio_med is_tourist is_general_service   
0        0          1          0            0          0                  0   
1        0          0          1            0          0                  0   
2        0          0          0            0          1                  0   
3        0          0          0            0          0                  0   
4        0          0          0            0          0                  0   

                                                                          \
  is_commerce is_commerce_spec is_grande_surface is_high_educ is_schools   
0           0                0                 0            0          0   
1           0                0                 0            0          0   
2           0                0                 0            0          0   
3           0                1                 0            0          0   
4           0                1                 0            0          0   

                
  is_transport  
0            0  
1            0  
2            0  
3            0  
4            0

### Merge the data

In [66]:
df = data.merge(data_cat,how='left',left_index=True, right_index=True)

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [67]:
df.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id,longitude,latitude,...,"(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)"
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801,7.321389,43.723333,...,0,0,0,0,0,0,0,0,0,0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005,7.278395,43.739469,...,1,0,0,0,0,0,0,0,0,0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005,7.278369,43.739746,...,0,0,1,0,0,0,0,0,0,0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801,7.269413,43.741919,...,0,0,0,0,0,1,0,0,0,0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801,7.258674,43.742321,...,0,0,0,0,0,1,0,0,0,0


### Label iris with tramway line

In [68]:
iris_t1 = pd.read_excel('../raw_data/iris_T1.xlsx')

In [69]:
iris_t1.head(3)

,iris_id,NOM_IRIS
0,60881304,Saint-Sylvestre
1,60881503,Jeanne d'Arc-Fuon-Cauda
2,60881303,Gorbella


In [70]:
iris_t1['iris_id']=iris_t1['iris_id'].map(lambda x: str(0)+str(x))

In [71]:
list_irist1 = list(iris_t1['iris_id'])
list_irist1[20:24]

['060880102', '060880507', '060882201', '060880503']

In [72]:
len(list_irist1)

35

In [73]:
df['iris_with_t1']=df['iris_id'].map(lambda x: 1 if x in list_irist1 else 0)
pd.unique(df['iris_with_t1'])

array([0, 1])

In [74]:
df.iris_with_t1.value_counts()

0    6143
1    5714
Name: iris_with_t1, dtype: int64

In [75]:
df.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris_id,longitude,latitude,...,"(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",iris_with_t1
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,060880801,7.321389,43.723333,...,0,0,0,0,0,0,0,0,0,0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,060881005,7.278395,43.739469,...,0,0,0,0,0,0,0,0,0,0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,060881005,7.278369,43.739746,...,0,1,0,0,0,0,0,0,0,0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,060883801,7.269413,43.741919,...,0,0,0,0,1,0,0,0,0,0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,060883801,7.258674,43.742321,...,0,0,0,0,1,0,0,0,0,0


## Dataset with important features

In [76]:
list(df.columns);

In [77]:
imp_feat = [ 'iris_id','iris_with_t1','AN', 'longitude', 'latitude',
            ('LAB', 'is_sport'), ('LAB', 'is_leasure'), ('LAB', 'is_medical'), ('LAB', 'is_socio_med'),
            ('LAB', 'is_tourist'), ('LAB', 'is_general_service'), ('LAB', 'is_commerce'), ('LAB', 'is_commerce_spec'),
            ('LAB', 'is_grande_surface'), ('LAB', 'is_high_educ'), ('LAB', 'is_schools'), ('LAB', 'is_transport'),
            'Q_XY_acceptable', 'Q_XY_bonne', 'Q_XY_mauvaise', 'LAB', 'CLASSE']

In [78]:
df_main = df[imp_feat]

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## Main datasert

In [79]:
df_main.head()

,iris_id,iris_with_t1,AN,longitude,latitude,"(LAB, is_sport)","(LAB, is_leasure)","(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)",...,"(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,LAB,CLASSE
0,060880801,0,2018,7.321389,43.723333,0,1,0,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Centre équestre,Sport
1,060881005,0,2018,7.278395,43.739469,0,0,1,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Masseur kinésithérapeute D235,Santé
2,060881005,0,2018,7.278369,43.739746,0,0,0,0,1,...,0,0,0,0,0,0.0,1.0,0.0,Information touristique,Tourisme
3,060883801,0,2018,7.269413,43.741919,0,0,0,0,0,...,1,0,0,0,0,0.0,1.0,0.0,Agence immobilière,Services a
4,060883801,0,2018,7.258674,43.742321,0,0,0,0,0,...,1,0,0,0,0,0.0,0.0,1.0,Agence immobilière,Services a


In [80]:
df_main.shape

(11857, 22)

In [81]:
df_main.to_csv('../raw_data/bpe_t1_clean.csv')

### Import dataset

In [82]:
df_main.to_csv('../raw_data/bpe_t1_clean.csv')

### Aggregation par iris de different services a la population

In [83]:
def count_sport(cat):
    return cat.map(lambda x: 1 if x in  cat_sport else 0).sum()

def count_leasure(cat):
    return cat.map(lambda x: 1 if x in  cat_leasure else 0).sum()

def count_medical(cat):
    return cat.map(lambda x: 1 if x in  cat_medical else 0).sum()

def count_socio_med(cat):
    return cat.map(lambda x: 1 if x in  cat_socio_med else 0).sum()

def count_tourist(cat):
    return cat.map(lambda x: 1 if x in  cat_tourist else 0).sum()

def count_general_service(cat):
    return cat.map(lambda x: 1 if x in  cat_general_service else 0).sum()

def count_commerce(cat):
    return cat.map(lambda x: 1 if x in  cat_commerce else 0).sum()

def count_commerce_spec(cat):
    return cat.map(lambda x: 1 if x in  cat_commertce_x else 0).sum()

def count_grande_surface(cat):
    return cat.map(lambda x: 1 if x in  cat_grande_surface else 0).sum()

def count_high_educ(cat):
    return cat.map(lambda x: 1 if x in  cat_high_educ else 0).sum()

def count_schools(cat):
    return cat.map(lambda x: 1 if x in  cat_schools else 0).sum()

def count_transport(cat):
    return cat.map(lambda x: 1 if x in  cat_transport else 0).sum()
#def count_iris_t1(cat):
#    return cat.map(lambda x: 1 if x in list_irist1 esle 0).mean()

In [84]:
df_main_agg = df_main.groupby('iris_id').agg({
    'LAB':[count_sport, count_leasure, count_medical, count_socio_med, 
           count_tourist,count_general_service,count_commerce,count_commerce_spec,
           count_grande_surface,count_high_educ,count_schools,count_transport]
})
df_main_agg

LAB                                              \
          count_sport count_leasure count_medical count_socio_med   
iris_id                                                             
060880101           0             1            27               1   
060880102           0             1            17               0   
060880103           7             0            17               1   
060880201           2             2            68               1   
060880202           1             0            21               0   
...               ...           ...           ...             ...   
060883301           0             0             0               0   
060883501           0             0             0               0   
060883701           0             0             1               1   
060883801           0             1            16               1   
061590101           1             0             0               0   

                                                              \
          count_tourist count_general_service count_commerce   
iris_id                                                        
060880101             7                     2             35   
060880102             2                     0             40   
060880103             3                     0             37   
060880201             4                     1             50   
060880202             8                     1             20   
...                 ...                   ...            ...   
060883301             0                     0              0   
060883501             0                     0              0   
060883701             0                     0              0   
060883801             2                     0              3   
061590101             0                     0              0   

                                                                    \
          count_commerce_spec count_grande_surface count_high_educ   
iris_id                                                              
060880101                 190                    6               2   
060880102                 226                    2               0   
060880103                 115                    3               0   
060880201                 121                   11               0   
060880202                  82                   10               2   
...                       ...                  ...             ...   
060883301                   0                    0               0   
060883501                   0                    0               0   
060883701                   0                    0               0   
060883801                  31                    1               0   
061590101                   0                    1               0   

                                         
          count_schools count_transport  
iris_id                                  
060880101             4               1  
060880102             0               1  
060880103             2               2  
060880201             6               7  
060880202             3               7  
...                 ...             ...  
060883301             1               0  
060883501             1               0  
060883701             2               0  
060883801             7               6  
061590101             0               0  

[110 rows x 12 columns]

In [85]:
df_main_agg.index

Index(['060880101', '060880102', '060880103', '060880201', '060880202',
       '060880203', '060880204', '060880205', '060880206', '060880301',
       ...
       '060882805', '060882902', '060883002', '060883101', '060883102',
       '060883301', '060883501', '060883701', '060883801', '061590101'],
      dtype='object', name='iris_id', length=110)

In [86]:
#df_main_agg['iris_with_t1']=
df_main_agg['iris_with_t1']=df_main_agg.index.map(lambda x: 1 if x in list_irist1 else 0)

### Aggregated main dataset 

In [87]:
df_main_agg.head(6)

LAB                                              \
          count_sport count_leasure count_medical count_socio_med   
iris_id                                                             
060880101           0             1            27               1   
060880102           0             1            17               0   
060880103           7             0            17               1   
060880201           2             2            68               1   
060880202           1             0            21               0   
060880203           0             0            33               0   

                                                              \
          count_tourist count_general_service count_commerce   
iris_id                                                        
060880101             7                     2             35   
060880102             2                     0             40   
060880103             3                     0             37   
060880201             4                     1             50   
060880202             8                     1             20   
060880203             4                     0             20   

                                                                    \
          count_commerce_spec count_grande_surface count_high_educ   
iris_id                                                              
060880101                 190                    6               2   
060880102                 226                    2               0   
060880103                 115                    3               0   
060880201                 121                   11               0   
060880202                  82                   10               2   
060880203                 116                    9               1   

                                        iris_with_t1  
          count_schools count_transport               
iris_id                                               
060880101             4               1            0  
060880102             0               1            1  
060880103             2               2            1  
060880201             6               7            1  
060880202             3               7            0  
060880203            11               2            0

In [88]:
df_main_agg.shape

(110, 13)

In [89]:
df_main_agg.to_csv('../raw_data/bpe_t1_agg_iris_clean.csv')

# second dataset T2

In [90]:
data2 = pd.read_csv('../raw_data/BPE_T2.csv')

In [91]:
data2.head()

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,TC_2017,cadr_2015,cadr_2016,cadr_2017,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
3,4,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
4,5,93,6,6088,06088_0301,2018,A203,1045176.2,6299125.70,Bonne,...,6.962,13.958,-8.0,-12.207,1.366,-2.400,7.657,15.211,3,5


## Descriptive statistics

In [92]:
data2.shape

(11419, 40)

In [93]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11419 entries, 0 to 11418
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fid         11419 non-null  int64  
 1   REG         11419 non-null  int64  
 2   DEP         11419 non-null  int64  
 3   DEPCOM      11419 non-null  int64  
 4   DCIRIS      11419 non-null  object 
 5   AN          11419 non-null  int64  
 6   TYPEQU      11419 non-null  object 
 7   LAMBERT_X   11419 non-null  float64
 8   LAMBERT_Y   11419 non-null  float64
 9   QUALITE_XY  11419 non-null  object 
 10  LAB         11419 non-null  object 
 11  CLASSE      11419 non-null  object 
 12  DEPCOM_2    11419 non-null  int64  
 13  NOM_COM     11419 non-null  object 
 14  IRIS        11419 non-null  int64  
 15  iris_id     11419 non-null  int64  
 16  NOM_IRIS    11419 non-null  object 
 17  TYP_IRIS    11419 non-null  object 
 18  supkm2      11419 non-null  float64
 19  denskm2     11419 non-nul

In [94]:
data2['iris_id']=data2['DCIRIS'].map(lambda x: x[0:5]+x[6:])
data2['iris_id'].tail(4)

11415    061230101
11416    061230101
11417    061230101
11418    061230101
Name: iris_id, dtype: object

In [95]:
list(data2.columns)

['fid',
 'REG',
 'DEP',
 'DEPCOM',
 'DCIRIS',
 'AN',
 'TYPEQU',
 'LAMBERT_X',
 'LAMBERT_Y',
 'QUALITE_XY',
 'LAB',
 'CLASSE',
 'DEPCOM_2',
 'NOM_COM',
 'IRIS',
 'iris_id',
 'NOM_IRIS',
 'TYP_IRIS',
 'supkm2',
 'denskm2',
 'pop_2016',
 'ds_btp',
 'Agglo',
 'revenus',
 'bpe_index',
 'evol_15_24',
 '15_24_2016',
 '15_24_2017',
 'TC_2015',
 'TC_2016',
 'TC_2017',
 'cadr_2015',
 'cadr_2016',
 'cadr_2017',
 'cdi_2015',
 'cdi_2016',
 'cdi_2017',
 'ipondi_crs',
 'CL',
 'SC']

In [96]:
len(pd.unique(data2['TYPEQU']))

148

In [97]:
pd.unique(data2['DEPCOM'])

array([6123, 6088])

In [98]:
pd.unique(data2['ds_btp'])

array([1057.143, 2307.692, 1087.5  , 1554.348, 1157.303,      nan,
        229.299,  663.507,  375.   ,  308.333,  846.442, 1939.394,
       1233.333, 2357.143, 1869.565, 1076.596,  790.285, 1101.796,
       4217.391, 1253.676, 1978.102, 1046.025, 2470.588, 1705.882,
       1813.008, 1500.   , 1126.582,  852.861,  675.258, 1198.732,
        871.122,  980.769,  642.738,  766.355, 1596.154, 2142.857,
       1986.667, 1450.549, 1063.725, 1718.75 , 1485.437, 1807.339,
       1833.333,   93.525, 1906.977, 1450.   , 1440.789, 1527.473,
       1029.316, 1010.695, 1921.875, 1644.068, 1360.   , 1147.541,
        932.432,  739.91 ,  585.499, 1238.806, 1336.735, 1295.   ,
       1593.333,  280.569, 1493.151, 1541.502,  866.995])

In [99]:
data2['iris_type']=data2['TYP_IRIS'].map(lambda x: 1 if x =='A' else 0)

In [100]:
pd.unique(data2['TYP_IRIS'])

array(['H', 'A'], dtype=object)

In [101]:
data2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,cadr_2015,cadr_2016,cadr_2017,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC,iris_type
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0,0


## Coordinates conversion

In [102]:
# create coordinates input list

lambert2_x = list(data2['LAMBERT_X'])
lamber2_y = list(data2['LAMBERT_Y'])
coord2 = zip(lambert2_x,lamber2_y)

In [103]:
longitude = []
latitude = []
for x, y in coord2:
    pt = convertToWGS84Deg(x, y, Lambert93)
    longitude.append(pt.getX())
    latitude.append(pt.getY())

In [104]:
data2['longitude']=longitude
data2['latitude']=latitude

In [105]:
data2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,cadr_2017,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC,iris_type,longitude,latitude
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,0,7.178602,43.658636
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,0,7.178602,43.658636
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,0,7.178602,43.658636


## Encoding

In [106]:
pd.unique(data2['QUALITE_XY'])

array(['Bonne', 'Acceptable', 'Mauvaise'], dtype=object)

In [107]:
data2['QUALITE_XY'].value_counts()

Bonne         11162
Mauvaise        161
Acceptable       96
Name: QUALITE_XY, dtype: int64

In [108]:
quality_xy_enc = enc.transform(data2[['QUALITE_XY']])


In [109]:
quality_xy_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [110]:
data2["Q_XY_acceptable"], data2["Q_XY_bonne"], data2['Q_XY_mauvaise']=quality_xy_enc.T

In [111]:
data2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,cdi_2017,ipondi_crs,CL,SC,iris_type,longitude,latitude,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,0.796,36.487,2,0,0,7.178602,43.658636,0.0,1.0,0.0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,0.796,36.487,2,0,0,7.178602,43.658636,0.0,1.0,0.0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,0.796,36.487,2,0,0,7.178602,43.658636,0.0,1.0,0.0


## Categories of LAB

In [112]:
pd.unique(data2['LAB']);

In [113]:
data2_cat = data.agg({
    'LAB':[is_sport, is_leasure, is_medical, is_socio_med, 
           is_tourist,is_general_service,is_commerce,is_commerce_spec,
           is_grande_surface,is_high_educ,is_schools,is_transport]
})

In [114]:
data2_cat.isna().value_counts()

(LAB, is_sport)  (LAB, is_leasure)  (LAB, is_medical)  (LAB, is_socio_med)  (LAB, is_tourist)  (LAB, is_general_service)  (LAB, is_commerce)  (LAB, is_commerce_spec)  (LAB, is_grande_surface)  (LAB, is_high_educ)  (LAB, is_schools)  (LAB, is_transport)
False            False              False              False                False              False                      False               False                    False                     False                False              False                  11857
dtype: int64

## Merge datasets

In [115]:
df2 = data2.merge(data2_cat,how='left',left_index=True, right_index=True)
df2.head(3)

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,"(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)"
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,0,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,1,0,0,0,0,0,0,0,0,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,0,0,1,0,0,0,0,0,0,0


In [116]:
df2.iris_id

0        061230102
1        061230102
2        061230102
3        061230102
4        060880301
           ...    
11414    061230101
11415    061230101
11416    061230101
11417    061230101
11418    061230101
Name: iris_id, Length: 11419, dtype: object

## Label iris with tramway line 2

In [117]:
iris_t2 = pd.read_excel('../raw_data/IrisT2.xlsx')
list(iris_t2.columns);

In [118]:
iris_t2.dtypes

iris_id            int64
DEPCOM             int64
NOM_COM           object
IRIS               int64
NOM_IRIS          object
                  ...   
TE_pop_55_64     float64
TE_pop_65_79     float64
TE_pop_80plus    float64
ds_btp           float64
ipondi_crs       float64
Length: 230, dtype: object

In [119]:
iris_t2['iris_id']=iris_t2['iris_id'].apply(lambda x: str(x))

In [120]:
iris_t2.iris_id[7]

'60883001'

In [121]:
type(iris_t2.iris_id[8])

str

In [122]:
iris_t2.dtypes

iris_id           object
DEPCOM             int64
NOM_COM           object
IRIS               int64
NOM_IRIS          object
                  ...   
TE_pop_55_64     float64
TE_pop_65_79     float64
TE_pop_80plus    float64
ds_btp           float64
ipondi_crs       float64
Length: 230, dtype: object

In [123]:
iris_t2[['iris_id']]=iris_t2.iris_id.apply(lambda x: str(0)+x)

In [124]:
x=iris_t2.iris_id[8]

In [125]:
x

'060882906'

In [126]:
len(_)

9

In [127]:
list_irist2 = list(iris_t2['iris_id'])

In [128]:
list_irist2[9]

'060882905'

In [129]:
iris_t2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Columns: 230 entries, iris_id to ipondi_crs
dtypes: float64(125), int64(100), object(5)
memory usage: 124.1+ KB


In [130]:
iris_t2[['iris_id','NOM_COM']].to_csv('../raw_data/iris_t2.csv')

In [131]:
df2['iris_with_t2']=df2['iris_id'].map(lambda x: 1 if x in list_irist2 else 0)

In [132]:
df2['iris_with_t1']=df2['iris_id'].map(lambda x: 1 if x in list_irist1 else 0)

In [133]:
df2['iris_with_t1'].value_counts()

0    8077
1    3342
Name: iris_with_t1, dtype: int64

In [134]:
df2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,"(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",iris_with_t2,iris_with_t1
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,1,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,1,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,1,0,0,0,0,0,0,0,1,0


## Feature selection

In [135]:
df2['iris_type']=df2['TYP_IRIS']

In [136]:
df2.columns

Index([                        'fid',                         'REG',
                               'DEP',                      'DEPCOM',
                            'DCIRIS',                          'AN',
                            'TYPEQU',                   'LAMBERT_X',
                         'LAMBERT_Y',                  'QUALITE_XY',
                               'LAB',                      'CLASSE',
                          'DEPCOM_2',                     'NOM_COM',
                              'IRIS',                     'iris_id',
                          'NOM_IRIS',                    'TYP_IRIS',
                            'supkm2',                     'denskm2',
                          'pop_2016',                      'ds_btp',
                             'Agglo',                     'revenus',
                         'bpe_index',                  'evol_15_24',
                        '15_24_2016',                  '15_24_2017',
                           'TC_201

In [137]:
imp_feat2 = [ 'iris_id','iris_type','iris_with_t1','iris_with_t2','AN', 'longitude', 'latitude',
            ('LAB', 'is_sport'), ('LAB', 'is_leasure'), ('LAB', 'is_medical'), ('LAB', 'is_socio_med'),
            ('LAB', 'is_tourist'), ('LAB', 'is_general_service'), ('LAB', 'is_commerce'), ('LAB', 'is_commerce_spec'),
            ('LAB', 'is_grande_surface'), ('LAB', 'is_high_educ'), ('LAB', 'is_schools'), ('LAB', 'is_transport'),
            'Q_XY_acceptable', 'Q_XY_bonne', 'Q_XY_mauvaise', 'LAB', 'CLASSE']

In [138]:
df_main2=df2[imp_feat2]

/home/mchistya/.pyenv/versions/3.8.6/envs/trampact/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [139]:
df_main2.head(3)

,iris_id,iris_type,iris_with_t1,iris_with_t2,AN,longitude,latitude,"(LAB, is_sport)","(LAB, is_leasure)","(LAB, is_medical)",...,"(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,LAB,CLASSE
0,061230102,H,0,1,2018,7.178602,43.658636,0,1,0,...,0,0,0,0,0,0.0,1.0,0.0,Électricien,Services a
1,061230102,H,0,1,2018,7.178602,43.658636,0,0,1,...,0,0,0,0,0,0.0,1.0,0.0,Agence immobilière,Services a
2,061230102,H,0,1,2018,7.178602,43.658636,0,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Taxi,Transport


In [140]:
df_main2.shape

(11419, 24)

In [141]:
df_main2.to_csv('../raw_data/bpe_t2_clean.csv')

## Aggregation at iris level

In [138]:
df_main_agg2 = df_main2.groupby('iris_id').agg({
    'LAB':[count_sport, count_leasure, count_medical, count_socio_med, 
           count_tourist,count_general_service,count_commerce,count_commerce_spec,
           count_grande_surface,count_high_educ,count_schools,count_transport]
})
df_main_agg2

LAB                                              \
          count_sport count_leasure count_medical count_socio_med   
iris_id                                                             
060880101           0             1            27               1   
060880102           0             1            17               0   
060880103           6             0            17               1   
060880201           2             2            68               1   
060880202           1             0            21               0   
...               ...           ...           ...             ...   
061230107           0             0            41               0   
061230108           2             0             0               0   
061230109           1             0             0               0   
061230110           3             0             0               0   
061230111           1             0             0               0   

                                                              \
          count_tourist count_general_service count_commerce   
iris_id                                                        
060880101             7                     2             35   
060880102             2                     0             40   
060880103             3                     0             37   
060880201             4                     1             50   
060880202             8                     1             21   
...                 ...                   ...            ...   
061230107             3                     3             40   
061230108             0                     0              0   
061230109             0                     0              1   
061230110             0                     0              0   
061230111             0                     0              0   

                                                                    \
          count_commerce_spec count_grande_surface count_high_educ   
iris_id                                                              
060880101                 190                    6               2   
060880102                 226                    2               0   
060880103                 115                    3               0   
060880201                 121                   10               0   
060880202                  83                   11               2   
...                       ...                  ...             ...   
061230107                  71                   13               0   
061230108                   0                    0               0   
061230109                   1                    0               0   
061230110                   0                    1               0   
061230111                   0                    0               0   

                                         
          count_schools count_transport  
iris_id                                  
060880101             4               1  
060880102             0               1  
060880103             2               2  
060880201             4               7  
060880202             3               7  
...                 ...             ...  
061230107             6               4  
061230108             1               1  
061230109             1               0  
061230110             0               0  
061230111             0               0  

[100 rows x 12 columns]

In [139]:
len(df_main_agg2.index)

100

In [140]:
df_main_agg2['iris_with_t1']=df_main_agg2.index.map(lambda x: 1 if x in list_irist1 else 0)

In [141]:
# Identify if T2 passes through the iris
df_main_agg2['iris_with_t2']=df_main_agg2.index.map(lambda x: 1 if x in list_irist2 else 0)

In [142]:
df_main_agg2.head(3)

LAB                                              \
          count_sport count_leasure count_medical count_socio_med   
iris_id                                                             
060880101           0             1            27               1   
060880102           0             1            17               0   
060880103           6             0            17               1   

                                                              \
          count_tourist count_general_service count_commerce   
iris_id                                                        
060880101             7                     2             35   
060880102             2                     0             40   
060880103             3                     0             37   

                                                                    \
          count_commerce_spec count_grande_surface count_high_educ   
iris_id                                                              
060880101                 190                    6               2   
060880102                 226                    2               0   
060880103                 115                    3               0   

                                        iris_with_t1 iris_with_t2  
          count_schools count_transport                            
iris_id                                                            
060880101             4               1            0            1  
060880102             0               1            1            1  
060880103             2               2            1            1

In [143]:
df_main_agg2.shape

(100, 14)

In [144]:
df_main_agg2.to_csv('../raw_data/bpe_t2_agg_iris_clean.csv')